In [1]:
import pandas as pd

In [2]:
#버퍼안에 속한 50m 격자 정보 불러옴
buffer = pd.read_csv('버퍼안 50m격자.csv')
buffer.head(1)

,cell_50_id,cell_250_i,x_katech,y_katech,gu_nm,hj_dong_nm,역
0,59733232,다바91bb15ba,347202,415343,중구,목동,중촌


In [31]:
#11.30 파일 데이터 크기가 수깁기가 넘어서거나 계속 정제하면 램부족등으로 재부팅 해줘야함
#특정달 15일 데이터
#data = pd.read_csv('DAEJEON_20190101_20190115.csv')#1월
#data = pd.read_csv('DAEJEON_20190201_20190215.csv')#2월
#data = pd.read_csv('DAEJEON_20190301_20190315.csv')#3월
#data = pd.read_csv('DAEJEON_20190401_20190415.csv')#4월
#data = pd.read_csv('DAEJEON_20190501_20190515.csv')#5월
#data = pd.read_csv('DAEJEON_20190601_20190615.csv')#6월
#data = pd.read_csv('DAEJEON_20190701_20190715.csv')#7월
#data = pd.read_csv('DAEJEON_20190801_20190815.csv')#8월
#data = pd.read_csv('DAEJEON_20190901_20190915.csv')#9월
#data = pd.read_csv('DAEJEON_20191001_20191015.csv')#10월
#data = pd.read_csv('DAEJEON_20191101_20191115.csv')#11월
data = pd.read_csv('DAEJEON_20191201_20191215.csv')#12월
data.head()

,id,x,y,timezn_cd,m00,m10,m15,m20,m25,m30,...,f35,f40,f45,f50,f55,f60,f65,f70,total,etl_ymd
0,56173942,332602,410243,0,0,0,0,1,0,1,...,1,1,0,1,0,0,0,0,16,20191204
1,56173942,332602,410243,0,0,0,0,1,1,1,...,1,1,1,1,0,0,0,0,13,20191213
2,56173942,332602,410243,0,0,0,0,2,1,1,...,1,1,1,1,1,0,0,0,17,20191215
3,56173942,332602,410243,0,0,0,1,1,1,1,...,0,0,1,0,0,0,0,0,10,20191202
4,56173942,332602,410243,0,0,0,1,1,1,1,...,0,1,1,1,0,0,0,0,16,20191206


In [32]:
#특정달 16~말일 데이터
#data2 = pd.read_csv('DAEJEON_20190116_20190131.csv')#1월
#data2 = pd.read_csv('DAEJEON_20190216_20190228.csv')#2월
#data2 = pd.read_csv('DAEJEON_20190316_20190331.csv')#3월
#data2 = pd.read_csv('DAEJEON_20190416_20190430.csv')#4월
#data2 = pd.read_csv('DAEJEON_20190516_20190531.csv')#5월
#data2 = pd.read_csv('DAEJEON_20190616_20190630.csv')#6월
#data2 = pd.read_csv('DAEJEON_20190716_20190731.csv')#7월
#data2 = pd.read_csv('DAEJEON_20190816_20190831.csv')#8월
#data2 = pd.read_csv('DAEJEON_20190916_20190930.csv')#9월
#data2 = pd.read_csv('DAEJEON_20191016_20191031.csv')#10월
#data2 = pd.read_csv('DAEJEON_20191116_20191130.csv')#11월
data2 = pd.read_csv('DAEJEON_20191216_20191231.csv')#12월
data2.head()

,id,x,y,timezn_cd,m00,m10,m15,m20,m25,m30,...,f35,f40,f45,f50,f55,f60,f65,f70,total,etl_ymd
0,56173942,332602,410243,0,0,0,0,0,2,2,...,0,1,3,1,1,0,0,0,17,20191217
1,56173942,332602,410243,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,8,20191226
2,56173942,332602,410243,0,0,0,1,1,0,1,...,0,1,1,1,1,0,0,0,16,20191231
3,56173942,332602,410243,0,0,0,1,1,0,1,...,0,1,1,1,0,0,0,0,19,20191228
4,56173942,332602,410243,0,0,0,1,1,1,0,...,1,0,1,0,0,0,0,0,18,20191222


In [33]:
#분석을 위해 하나의 파일로 합침
month_data = pd.concat([data, data2])

In [34]:
#필요없는 속성값만 지정해서 제거
del month_data['x']
del month_data['y']
del month_data['etl_ymd']
del month_data['timezn_cd']

In [35]:
#피봇 테이블 이용해 각 id별 유동인구 데이터 합침
pdf1 = pd.pivot_table(month_data,                
                     index = 'id',          
                     aggfunc = 'sum')

In [36]:
#조인해서 격자로 된 유동인구 데이터중 버퍼에 속한 격자 데이터만 가져옴
buffer_join = pd.merge(buffer,pdf1, left_on='cell_50_id', right_on='id', how='left')

In [37]:
#결측값은 유동인구 측정이 안되는 부분이니 0으로 처리 
buffer_join = buffer_join.fillna(0)


In [38]:
buffer_join['20대미만_남'] = buffer_join['m00'] +buffer_join['m10'] + buffer_join['m15']
buffer_join['20대_남'] = buffer_join['m20'] + buffer_join['m25']
buffer_join['30대_남'] = buffer_join['m30'] + buffer_join['m35']
buffer_join['40대_남'] = buffer_join['m40'] + buffer_join['m45']
buffer_join['50대_남'] = buffer_join['m50'] + buffer_join['m55']
buffer_join['60대이상_남'] = buffer_join['m60'] + buffer_join['m65'] +buffer_join['m70']

buffer_join['20대미만_여'] = buffer_join['f00'] +buffer_join['f10'] + buffer_join['f15']
buffer_join['20대_여'] = buffer_join['f20'] + buffer_join['f25']
buffer_join['30대_여'] = buffer_join['f30'] + buffer_join['f35']
buffer_join['40대_여'] = buffer_join['f40'] + buffer_join['f45']
buffer_join['50대_여'] = buffer_join['f50'] + buffer_join['f55']
buffer_join['60대이상_여'] = buffer_join['f60'] + buffer_join['f65'] +buffer_join['f70']


In [39]:
buffer_join['총합_남'] = buffer_join['20대미만_남']+buffer_join['20대_남']+buffer_join['30대_남']+buffer_join['40대_남']+buffer_join['50대_남']+buffer_join['60대이상_남']
buffer_join['총합_여'] = buffer_join['20대미만_여']+buffer_join['20대_여']+buffer_join['30대_여']+buffer_join['40대_여']+buffer_join['50대_여']+buffer_join['60대이상_여']


In [40]:
#필요없는 열들 삭제
del buffer_join['m00']
del buffer_join['m10']
del buffer_join['m15']
del buffer_join['m20']
del buffer_join['m25']
del buffer_join['m30']
del buffer_join['m35']
del buffer_join['m40']
del buffer_join['m45']
del buffer_join['m50']
del buffer_join['m55']
del buffer_join['m60']
del buffer_join['m65']
del buffer_join['m70']

del buffer_join['f00']
del buffer_join['f10']
del buffer_join['f15']
del buffer_join['f20']
del buffer_join['f25']
del buffer_join['f30']
del buffer_join['f35']
del buffer_join['f40']
del buffer_join['f45']
del buffer_join['f50']
del buffer_join['f55']
del buffer_join['f60']
del buffer_join['f65']
del buffer_join['f70']

In [41]:
buffer_join.head()

,cell_50_id,cell_250_i,x_katech,y_katech,gu_nm,hj_dong_nm,역,total,20대미만_남,20대_남,...,50대_남,60대이상_남,20대미만_여,20대_여,30대_여,40대_여,50대_여,60대이상_여,총합_남,총합_여
0,59733232,다바91bb15ba,347202,415343,중구,목동,중촌,119178.0,3814.0,9637.0,...,13177.0,15954.0,3896.0,10451.0,8153.0,10644.0,10761.0,10116.0,65157.0,54021.0
1,58416741,다바86ba11ba,341802,411393,서구,가수원동,가수원,2562.0,59.0,264.0,...,327.0,305.0,37.0,159.0,161.0,246.0,302.0,210.0,1447.0,1115.0
2,58416742,다바86ba11ba,341802,411443,서구,가수원동,가수원,4774.0,125.0,447.0,...,582.0,604.0,85.0,312.0,294.0,450.0,558.0,412.0,2663.0,2111.0
3,58416743,다바86ba11ba,341802,411493,서구,가수원동,가수원,19099.0,771.0,2226.0,...,1930.0,2727.0,655.0,1757.0,1243.0,1265.0,1809.0,1267.0,11103.0,7996.0
4,58416744,다바86ba11ba,341802,411543,서구,가수원동,가수원,16134.0,649.0,1921.0,...,1622.0,2315.0,532.0,1485.0,1044.0,1077.0,1521.0,1057.0,9418.0,6716.0


In [42]:
#일 평균값으로 바꾼후 시간별 평균으로 나눔
buffer_join.iloc[:, 7:22] = buffer_join.iloc[:, 7:22]/31 #그 달의 월 값입력

buffer_join.iloc[:, 7:22] = buffer_join.iloc[:, 7:22]/24 #시간이니 고정

In [43]:
buffer_join.head()

,cell_50_id,cell_250_i,x_katech,y_katech,gu_nm,hj_dong_nm,역,total,20대미만_남,20대_남,...,50대_남,60대이상_남,20대미만_여,20대_여,30대_여,40대_여,50대_여,60대이상_여,총합_남,총합_여
0,59733232,다바91bb15ba,347202,415343,중구,목동,중촌,160.185484,5.126344,12.952957,...,17.711022,21.443548,5.236559,14.047043,10.958333,14.306452,14.463710,13.596774,87.576613,72.608871
1,58416741,다바86ba11ba,341802,411393,서구,가수원동,가수원,3.443548,0.079301,0.354839,...,0.439516,0.409946,0.049731,0.213710,0.216398,0.330645,0.405914,0.282258,1.944892,1.498656
2,58416742,다바86ba11ba,341802,411443,서구,가수원동,가수원,6.416667,0.168011,0.600806,...,0.782258,0.811828,0.114247,0.419355,0.395161,0.604839,0.750000,0.553763,3.579301,2.837366
3,58416743,다바86ba11ba,341802,411493,서구,가수원동,가수원,25.670699,1.036290,2.991935,...,2.594086,3.665323,0.880376,2.361559,1.670699,1.700269,2.431452,1.702957,14.923387,10.747312
4,58416744,다바86ba11ba,341802,411543,서구,가수원동,가수원,21.685484,0.872312,2.581989,...,2.180108,3.111559,0.715054,1.995968,1.403226,1.447581,2.044355,1.420699,12.658602,9.026882


In [44]:
buffer_join.to_excel('버퍼안 유동인구/12월 버퍼안 유동인구.xlsx')